In [3]:
import pandas as pd
from mrmr import mrmr_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.models import Sequential

In [4]:
train_data = pd.read_csv("C:\\123\\SRMAP\\Semester 5\\UROP\\\Train and Test dataset\\KDDTrain+.txt", header=None)
test_data = pd.read_csv("C:\\123\\SRMAP\\Semester 5\\UROP\\Train and Test dataset\\KDDTest+.txt", header=None)

In [5]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

train_data.columns = columns
test_data.columns = columns

# sanity check
train_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [6]:
# map normal to 0, all attacks to 1
train_attack = train_data.attack.map(lambda a: 0 if a == 'normal' else 1)
test_attack = test_data.attack.map(lambda a: 0 if a == 'normal' else 1)

#data_with_attack = df.join(is_attack, rsuffix='_flag')
train_data['attack_bit'] = train_attack
test_data['attack_bit'] = test_attack

# view the result
test_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level,attack_bit
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21,1
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21,1
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21,0
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15,1
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11,1


In [7]:
train_data['attack_bit'].value_counts()

attack_bit
0    67343
1    58630
Name: count, dtype: int64

In [8]:
train_data_attack = train_data[train_data['attack_bit']==1]
train_data_attack.shape

(58630, 44)

In [9]:
train_data_non_attack = train_data[train_data['attack_bit']==0]
train_data_non_attack.shape

(67343, 44)

In [10]:
train_data_non_attack_downsampled = train_data_non_attack.sample(train_data_attack.shape[0])
train_data_non_attack_downsampled.shape

(58630, 44)

In [11]:
train_data_balanced = pd.concat([train_data_non_attack_downsampled, train_data_attack])
train_data_balanced.shape

(117260, 44)

In [12]:
columns_to_drop = ['attack_bit', 'level', 'attack']

In [13]:
x_train = train_data_balanced.drop(columns=columns_to_drop, axis=1)
y_train = train_data_balanced['attack_bit']

numeric_columns1 = x_train.select_dtypes(include=['number']).columns
categorical_columns1 = x_train.select_dtypes(exclude=['number']).columns
X_categorical1 = pd.get_dummies(x_train[categorical_columns1], drop_first=True)
train = pd.concat([x_train[numeric_columns1], X_categorical1], axis=1)
train = train.astype(int)

In [14]:
x_test = test_data.drop(columns=columns_to_drop, axis=1)
y_test = test_data['attack_bit']

numeric_columns2 = x_test.select_dtypes(include=['number']).columns
categorical_columns2 = x_test.select_dtypes(exclude=['number']).columns
X_categorical2 = pd.get_dummies(x_test[categorical_columns2], drop_first=True)
test = pd.concat([x_test[numeric_columns2], X_categorical2], axis=1)
test = test.astype(int)
test.head(20)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,15,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,267,14515,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,0,1022,387,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
7,0,129,174,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,327,467,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9,0,26,157,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
selected_features = mrmr_classif(train, y_train, 5) # Select 10 features
X_selected = train[selected_features]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.13it/s]


In [16]:
print(X_selected)

        flag_SF  dst_host_srv_count  logged_in  same_srv_rate   
101009        1                 255          1              1  \
13914         1                 255          1              1   
115574        1                 255          1              1   
125859        1                 255          1              1   
115849        1                 156          1              1   
...         ...                 ...        ...            ...   
125958        0                   4          0              0   
125964        0                  12          0              0   
125966        0                  13          0              0   
125968        0                  25          0              0   
125971        0                   8          0              0   

        dst_host_srv_serror_rate  
101009                         0  
13914                          0  
115574                         0  
125859                         0  
115849                         0  
...      

In [9]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_selected)
# clf = MLPClassifier(hidden_layer_sizes=(64,32), random_state=42)

In [11]:
 #clf.fit(X_train, y_train)
# num_epochs = 5 # Specify the number of "epochs" you want
# for epoch in range(num_epochs):
#     clf.fit(X_train, y_train)
# You can perform any necessary evaluations or updates here

In [1]:
# X_test_selected = test[selected_features]
# X_test = scaler.transform(X_test_selected)
# accuracy = model.score(X_test, y_test)
# print(f"Accuracy: {accuracy}")

NameError: name 'test' is not defined

In [17]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_selected)

# Create a Sequential model with GRU layers and an RBF layer
model = Sequential()

# Add GRU layers
model.add(GRU(64, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(32, activation='tanh'))

# Add an RBF layer
model.add(Dense(1, activation='linear')) # You can adjust the number of units as needed

# Compile the model
#model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Fit the model
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

In [18]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

# Reshape X_train for compatibility with GRU
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [19]:
# Fit the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
3298/3298 [==============================] - 27s 7ms/step - loss: 0.2579 - accuracy: 0.8895 - precision: 0.9199 - recall: 0.8231 - val_loss: 0.3152 - val_accuracy: 0.8367 - val_precision: 1.0000 - val_recall: 0.8367
Epoch 2/5
3298/3298 [==============================] - 25s 8ms/step - loss: 0.2247 - accuracy: 0.8957 - precision: 0.9259 - recall: 0.8320 - val_loss: 0.2508 - val_accuracy: 0.8618 - val_precision: 1.0000 - val_recall: 0.8618
Epoch 3/5
3298/3298 [==============================] - 30s 9ms/step - loss: 0.2223 - accuracy: 0.9103 - precision: 0.9487 - recall: 0.8439 - val_loss: 0.1500 - val_accuracy: 0.9076 - val_precision: 1.0000 - val_recall: 0.9076
Epoch 4/5
3298/3298 [==============================] - 27s 8ms/step - loss: 0.2106 - accuracy: 0.9158 - precision: 0.9458 - recall: 0.8598 - val_loss: 0.2571 - val_accuracy: 0.8911 - val_precision: 1.0000 - val_recall: 0.8911
Epoch 5/5
3298/3298 [==============================] - 30s 9ms/step - loss: 0.2043 - accuracy: 0

In [38]:
X_test_selected = test[selected_features]
X_test = scaler.fit_transform(test)  # Apply the same scaler used for training data
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)  # Reshape for compatibility with the model

In [39]:
evaluation = model.evaluate(X_test, y_test)

705/705 [==============================] - 10s 14ms/step - loss: 2.2543 - accuracy: 0.5716 - precision: 0.7940 - recall: 0.3342


In [40]:
loss = evaluation[0]
binary_accuracy = evaluation[1]
precision = evaluation[2]
recall = evaluation[3]

In [42]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_selected)  # Get model predictions for the test data
y_pred_binary = (y_pred >= 0.5).astype(int)  # Convert probabilities to binary labels

# Generate a classification report
classification_rep = classification_report(y_test, y_pred_binary)

print(classification_rep)

705/705 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.43      1.00      0.60      9711
           1       0.00      0.00      0.00     12833

    accuracy                           0.43     22544
   macro avg       0.22      0.50      0.30     22544
weighted avg       0.19      0.43      0.26     22544

